Migrate CSV files data into Sql Database

In [0]:
dbutils.fs.mounts()

In [0]:
mount_path = "/mnt/j2dadlscontainer"
folder = "raw_datasets"
base_path = f"{mount_path}/{folder}"

In [0]:
# for name in dbutils.fs.ls(base_path):
#     print(name)

In [0]:
def get_file_paths(path):
    file_paths = []
    for name in dbutils.fs.ls(path): 
        file_paths.append(name[0])
    return file_paths

In [0]:
# get_file_paths(base_path)

In [0]:
jdbcHostname = "j2dtechnologies.database.windows.net"
jdbcPort = 1433
jdbcDatabase = "j2d_training_db"
user_name = dbutils.secrets.get(scope="j2dtech-secrets3", key="azure-sql-user-name")
user_password = dbutils.secrets.get(scope="j2dtech-secrets3", key="azure-sql-password")

jdbcUrl = f"jdbc:sqlserver://{jdbcHostname}:{jdbcPort};database={jdbcDatabase}"

connection_properties = {
  "user": user_name,
  "password": user_password,
  "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

In [0]:
def migrate_into_sql(base_path):
    file_paths = get_file_paths(base_path) # list of paths
    for file_path in file_paths:
        table_name = (file_path.split("/")[-1])
        if table_name.endswith(".csv"):
            df = (
                spark
                .read
                .format("csv")
                .option("header", "true")
                .option("inferSchema", True)
                .load(file_path)
            )

            # write to sql db
            try:
                (
                    df
                    .write
                    .format("jdbc")
                    .mode("overwrite")
                    .option("url", jdbcUrl)
                    .option("dbtable", table_name.replace(".csv", ""))
                    .options(**connection_properties)
                    .save()
                )
                print(f"Written {table_name} to Azure Sql as table")
            except Exception as e:
                print(e)

In [0]:
migrate_into_sql(base_path)